---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [93]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [30]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [31]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    file = open('university_towns.txt')
    list = []
    cur_state = ''
    for line in file:
        state_pos = line.find('[ed')
        region_pos = line.find('(')
        if state_pos != -1:
            cur_state = line[:state_pos].strip()
        elif region_pos != -1:
            region = line[:region_pos].strip()
            list.append((cur_state,region))
        else:
            list.append((cur_state,line.strip()))
    file.close()
    df = pd.DataFrame(list, columns=["State", "RegionName"])
    return df
df = get_list_of_university_towns()
# df[df.State=='Massachusetts']

You can compute the difference between a value and its predecessor in a series using df['diff'] = df['gdp'].diff()

use df['diff'] to create gdp_b which is a series with value 0 if GDP difference is less than 0 i.e declines and 1 if GDP difference is greater that 0 i.e increased. does this look familiar? Assignment 3 answer_ten()
<code>
QTR
2000q1    0
2000q2    1
2000q3    1
2000q4    1
2001q1    0
2001q2    1
.......
2015q1    1
2015q2    1
2015q3    1
2015q4    1
2016q1    1
2016q2    1
Name: gdp_b, dtype: int64
</code>
In order to find recession you will be looking for a sequence with 2 or more declines followed by 2 increases in GDP, i.e 0..0011.

This sequence can be found in many different ways, here are two

1. use a variable to store previous quarter gdp change (prev), and a list to keep track of the possible matching patterns (seq) while looping through gdp_b, the list will be appended and emptied depending on its length of and the current and the previous gdp change. The loop is exited when the second 1 in the sequence is found

2. convert the values of the series to a string i.e. ''.join(gdp_b.values) and use str.index() to find the index (ind) of the pattern "0011" then use str.rindex() to find the last occurrence of 1 before the pattern "0011" occurred, adding one to that will give you the position of the start recession. The recession end index will be ind-len(pattern)-1

In [32]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    def increase_two_consecutive_ArrayStartWith(arr):
        is_cur_consecu = False
        is_pre_consecu = False
        pos=0
        for inx in range(len(arr) - 1):
#       print(inx,arr[inx],is_pre_consecu,is_cur_consecu)
            if is_cur_consecu & is_pre_consecu:
                pos = arr[inx-2]
                break
            else:
                if arr[inx+1] - arr[inx] == 1:
                    is_pre_consecu = is_cur_consecu
                    is_cur_consecu = True
                else:
                    is_cur_consecu = False
        return pos   
    df = pd.read_excel('gdplev.xls',skiprows=7, parse_cols=[4,5,6],
                      names=['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars'])
    start = df[df.Quarter == '2000q1'].index[0]
    df = df.ix[start:]
    changes = df['GDP in billions of chained 2009 dollars'].pct_change()
#     df['diff'] = df['gdp'].diff()
    res_pos = changes[changes<0].index.values
    p = increase_two_consecutive_ArrayStartWith(res_pos)
    return df.ix[p, 'Quarter']
get_recession_start()

'2008q3'

In [33]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    def increase_two_consecutive_ArrayStartWith(arr):
        is_cur_consecu = False
        is_pre_consecu = False
        pos=0
        for inx in range(len(arr) - 1):
#             print(inx,arr[inx],is_pre_consecu,is_cur_consecu)
            if is_cur_consecu & is_pre_consecu:
                pos = arr[inx-1]
                break
            else:
                if arr[inx+1] - arr[inx] == 1:
                    is_pre_consecu = is_cur_consecu
                    is_cur_consecu = True
                else:
                    is_cur_consecu = False
        return pos   
    df = pd.read_excel('gdplev.xls',skiprows=7, parse_cols=[4,5,6],
                      names=['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars'])
    res_start = get_recession_start()
    start = df[df.Quarter == res_start].index[0]
    df = df.ix[start:]
    changes = df['GDP in billions of chained 2009 dollars'].pct_change()
#     df['diff'] = df['gdp'].diff()
#     print(changes[changes>0])
    res_pos = changes[changes>0].index.values
#     print(res_pos)
    p = increase_two_consecutive_ArrayStartWith(res_pos)
#     print(p)
    return df.ix[p, 'Quarter']
get_recession_end()

'2009q4'

In [34]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls',skiprows=7, parse_cols=[4,5,6],
                      names=['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars'])
    res_start = get_recession_start()
    start = df[df.Quarter == res_start].index[0]
    res_end = get_recession_end()
    end = df[df.Quarter == res_end].index[0]
    df = df.ix[start:end]
    bottom_pos = df['GDP in billions of chained 2009 dollars'].argmin()
    return df.ix[bottom_pos, 'Quarter']
get_recession_bottom()

'2009q2'

In [35]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
# http://stackoverflow.com/questions/41661451/pandas-compute-the-half-yearly-average-of-monthly-collected-house-sales
    df = pd.read_csv("City_Zhvi_AllHomes.csv")
    df['State'] = df['State'].map(states)
    columns = ["State","RegionName"]
    identifier = df[columns]
    hos = df[df.columns[6:]]
    df = None
    hos.columns = pd.to_datetime(hos.columns,format='%Y-%m')
    columns_2000 = hos.columns[hos.columns >= '2000-01-01']
    hos2000 = hos[columns_2000]
    hos=None
    hos2000_Q_mean = hos2000.resample('Q',axis=1).mean()
    hos2000_Q_mean = hos2000_Q_mean.rename(columns=lambda x: '{:}q{:}'.format(x.year, x.quarter))
    result = pd.concat([identifier, hos2000_Q_mean], axis=1)
    return result.set_index(columns)
# convert_housing_data_to_quarters().info()
convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q1"]

207066.66666666666

In [94]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    start = get_recession_start()
    bottom = get_recession_bottom()
    
    hourse_prices = convert_housing_data_to_quarters()
#     hourse_prices.columns = pd.to_datetime(hourse_prices.columns)
#     cond = hourse_prices.columns[(hourse_prices.columns >= start) & (hourse_prices.columns <= end)]
    hourse_prices = hourse_prices[[start, bottom]]
    hourse_prices = hourse_prices.sort_index()
    university_towns = get_list_of_university_towns()
    merged = pd.merge(hourse_prices.reset_index(), university_towns, on=['State','RegionName'], how='outer', indicator=True)
    merged['PriceRatio'] = merged[start].div(merged[bottom])
    merged = merged.set_index(["State","RegionName"])
    univeristy = merged[merged._merge == 'both']
    non_university = merged[merged._merge == 'left_only']
    stat, p=ttest_ind(univeristy['PriceRatio'].dropna(), non_university['PriceRatio'].dropna())
    return (p<0.01, p, ["university town", "non-university town"][univeristy['PriceRatio'].mean()>non_university['PriceRatio'].mean()])

run_ttest()

(True, 0.005496427353694603, 'university town')

In [76]:
#     start = get_recession_start()
#     bottom = get_recession_bottom()
    
#     hourse_prices = convert_housing_data_to_quarters()
# #     hourse_prices.columns = pd.to_datetime(hourse_prices.columns)
# #     cond = hourse_prices.columns[(hourse_prices.columns >= start) & (hourse_prices.columns <= bottom)]
#     hourse_prices = hourse_prices[[start, bottom]]
#     hourse_prices = hourse_prices.sort_index()
#     university_towns = get_list_of_university_towns()
#     merged = pd.merge(hourse_prices.reset_index(), university_towns, on=['State','RegionName'], how='outer', indicator=True)
#     merged['PriceRatio'] = merged[start].div(merged[bottom])
#     merged = merged.set_index(["State","RegionName"])
#     univeristy = merged[merged._merge == 'both']
#     non_university = merged[merged._merge == 'left_only']
#     stat, p=stats.ttest_ind(univeristy['PriceRatio'].dropna(), non_university['PriceRatio'].dropna())
    
#     hourse_prices.index.isin(university).sum()
    
    

In [90]:
# merged = pd.merge(hourse_prices.reset_index(), university_towns, on=['State','RegionName'], how='outer', indicator=True)
# univeristy = merged[merged._merge == 'both']
# non_university = merged[merged._merge == 'left_only']
# print(merged.shape, university.shape, non_university.shape)
# merged[merged._merge == 'right_only']

In [54]:

# hourse_prices.loc[('Illinois', 'Chicago')]
# hourse_prices.loc['Illinois', 'Chicago']
# hourse_prices.loc['Alabama', ]